In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")
sys.path.append(".")
import copy
import torch
import numpy as np
import pickle
from datetime import datetime
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

import robust_value_approx.utils as utils
import plotting_utils
import slip_utils

In [32]:
vf, slip = slip_utils.get_value_function(torch.Tensor([0., 0., 0.]), N=3)

Progress: [########################################] 100.0%
8 hybrid modes created


In [36]:
xf = torch.Tensor([2.5, 1.5, 0.])
xtraj = xf.type(vf.dtype).unsqueeze(1).repeat(1, vf.N-1)
vf.set_traj(xtraj=xtraj)

x0 = torch.Tensor([0., 1., 3.]).type(vf.dtype)
V = vf.get_value_function()
(x_traj, u_traj, alpha_traj) = vf.sol_to_traj(x0, *V(x0)[1:])

print(u_traj)

tensor([[0.4181, 0.4119, 0.4119]], dtype=torch.float64)


In [ ]:
x0_slip = torch.cat((x0, torch.Tensor([0.]).type(vf.dtype))).squeeze().numpy()
theta_step = u_traj.squeeze().numpy()

slip_apex_x = [x0[0]]
slip_apex_y = [x0[1]]
slip_apex_xdot = [x0[2]]
for theta in theta_step[:-1]:
    (next_x, next_y, next_xdot, _) = slip.apex_map(slip_apex_x[-1], slip_apex_y[-1], slip_apex_xdot[-1], theta)
    slip_apex_x.append(next_x)
    slip_apex_y.append(next_y)
    slip_apex_xdot.append(next_xdot)

sol = slip.simulate(x0_slip, theta_step)
slip_x = []
slip_y = []
for step in range(int(len(sol)/2)):
    slip_x.extend(sol[2 * step].y[0])
    slip_y.extend(sol[2 * step].y[1])
    slip_r = np.array(sol[2 * step + 1].y[0])
    slip_theta = np.array(sol[2 * step + 1].y[1])
    slip_x_foot = np.array(sol[2 * step + 1].y[4])
    slip_x.extend(list(slip_x_foot - slip_r * np.sin(slip_theta)))
    slip_y.extend(list(slip_r * np.cos(slip_theta)))
    
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=[xf[0]],
    y=[xf[1]],
    mode='markers',
    name='Goal',
    marker=dict(
        size=15,
        color=3),
))
fig.add_trace(go.Scatter(
    x=slip_x,
    y=slip_y,
    name="SLIP"))
fig.add_trace(go.Scatter(
    x=slip_apex_x,
    y=slip_apex_y,
    mode='markers',
    name='SLIP Apex',
    marker=dict(
        size=10,
        color=4),
))
fig.add_trace(go.Scatter(
    x=x_traj[0,:],
    y=x_traj[1,:],
    mode='markers',
    name='PL-model',
    marker=dict(
        size=10,
        color=3),
))
fig.show()